# Oppgave 3

In [1]:
from scipy import ndimage
from skimage import io,util,color
from IPython import display
import numpy
import sys
import math

import ipywidgets as widgets
from ipywidgets import interactive
from IPython.display import display, clear_output

import matplotlib.pyplot as plt

# Our imports
import skimage
import scipy
from skimage import util,color,exposure,io
from scipy import fftpack

%matplotlib inline

Funksjon for transformering til frekvensplanet

In [2]:
#Transformerer til frekvensplanet
def fourrier(im):
    # Retrieve the image dimensions
    height,width = im.shape

    # Create an image to store the transformed image
    image_trans = numpy.zeros(im.shape)
    # Transform each pixel in the image by multiplying with -1**(i+j) (see 
    # slides/book) to translate the fourier transformed image to height/2 width/2 
    # i.e. the center of the image
    for i in range(height):
      for j in range(width):
        image_trans[i,j] = im[i,j]*((-1)**(i+j))

    # Perform the fourier transformation
    ft = fftpack.fft2(image_trans)

    # Get the fourier spectrum by finding the length of the complex number vectors
    ft = numpy.absolute(ft)

    # Log transformation (see own lecture)
    out = numpy.zeros(im.shape)
    for i in range(0, im.shape[0]):
      for j in range(0, im.shape[1]):
        out[i,j] = numpy.log(1+ft[i,j])

    return out

Vil se hvordan ett bilde som kun har ett sinus signal i vertikal retning ser ut i frekvens domenet, sånn at jeg vet hva som skal filtreres vekk

In [10]:
def showSinusInFreq(freq):
    sinusImage = numpy.zeros((256,256))
    freq = (freq*math.pi)/256

    for x in range(256):
        for y in range(256):
            sinusImage[x,y] = math.sin(x*freq) 

    io.imshow_collection([sinusImage,fourrier(sinusImage)], cmap='gray')
    io.show()

interactive(showSinusInFreq, freq=(1,256))

interactive(children=(IntSlider(value=128, description='freq', max=256, min=1), Output()), _dom_classes=('widg…

Som vi kan se legger frekvensen seg som en prikk en plass langs vertikal akse, sentrert horisontalt. Klarer ikke finne noen sammenheng om priken er plassert over eller under midten, men når frekvensen øker flytter prikken seg lenger vekk fra midten.

Min plan nå er å finne den frekvensen som er nærmest den frekvensen i bildet ved å sjekke differansen mellom genererte sinus bilder og saturn bildet, til jeg finner den hvor frekvensen er nærmest den i bildet. Så lar jeg filteret være det inverse av den nærmeste bildet i frekvens domenet.

# OBS ikke kjør denne dersom du ikke har god tid
(det fungerte ikke uansett)

In [4]:
def plotHistogram(hist):
    ind = numpy.arange(len(hist))  

    fig, ax = plt.subplots()
    rects1 = ax.bar(ind, hist)

    plt.show()

    
inputImage = widgets.Dropdown(
    options={
        'Saturn Med støy':'cassini-interference.png'
        },
    description='Bilde: ',
    disabled=True,
)
    
def findFrequencyImage(sourceImage):
    im = io.imread(sourceImage, as_gray=True)
    
    width, height = im.shape
    
    diff = numpy.zeros(height)
    
    minDiff = math.inf
    minFreq = 0
    
    for freq0 in range(height):
        
        sinusImage = numpy.zeros(im.shape)
        freq = (freq0*math.pi)/height

        for x in range(width):
            for y in range(height):
                sinusImage[x,y] = math.sin(x*freq) 
        
        diffImage = im - 0.5*sinusImage
        difference = numpy.sum(diffImage)
        if(difference < minDiff):
            minFreq = freq0
            minDiff = difference
            
        diff[freq0] = difference
    
    sinusImage = numpy.zeros(im.shape)
    freq = (minFreq*math.pi)/height

    for x in range(width):
        for y in range(height):
            sinusImage[x,y] = 0.5 * math.sin(x*freq) 
    
    
    plotHistogram(diff)
    io.imshow_collection([im, sinusImage], cmap='gray')

interactive(findFrequencyImage, sourceImage = inputImage)

interactive(children=(Dropdown(description='Bilde: ', disabled=True, options={'Saturn Med støy': 'cassini-inte…

Dette innser jeg nå ikke vil fungere da, det bildet med mest hvitt innhold vil være det som matcher orginal bildet best.

Jeg vil helle da generere filteret manuelt til jeg finner de verdiene som passer best til å fjerne støy.

In [8]:
def fourrierFilter(image,cutoff):
    # Read image, convert to gray, convert to float
    im = util.img_as_float(color.rgb2gray(io.imread(image)))

    # Retrieve the image dimensions
    height,width = im.shape
    
    #finding P and Q
    padded_height = height*2
    padded_width = width*2
    
    #Padding image width 0s
    padded = numpy.zeros((padded_height,padded_width))
    padded[0:height,0:width] = im

    # Create an image to store the transformed image
    image_trans = numpy.zeros(padded.shape)
    # Transform each pixel in the image by multiplying with -1**(i+j) (see 
    # slides/book) to translate the fourier transformed image to height/2 width/2 
    # i.e. the center of the image
    for i in range(padded_height):
      for j in range(padded_width):
        image_trans[i,j] = padded[i,j]*((-1)**(i+j))

    # Perform the fourier transformation
    fourier_transform = fftpack.fft2(image_trans)



    #lage filter 
    filtr = numpy.zeros(padded.shape)
    center_w = padded_width/2
    center_h = padded_height/2
    for x in range(padded_height):
        for y in range(padded_width):
            filtr[x,y] = 0 if ((math.fabs(y - center_w) < 5) and (math.fabs(x - center_h) > cutoff) and (math.fabs(x - center_h) < (cutoff + 200))) else 1
    
    #filtrerer den fourier transformerte
    filtered = filtr*fourier_transform    
    
    #transformerer med invers fourier
    filtered = fftpack.ifft2(filtered)
    
    #translerer tilbake
    filtered_translated = numpy.zeros(padded.shape)

    for i in range(padded_height):
        for j in range(padded_width):
            filtered_translated[i,j] = filtered[i,j]*((-1)**(i+j))
    
    #fjerner padding
    filtered_cropped = filtered_translated[0:width,0:height]
    
    # Show the result
    io.imshow_collection([numpy.clip(im,0,1),numpy.clip(filtered_cropped,0,1),filtr], cmap='gray')
    io.show()
    
interactive(fourrierFilter,image=inputImage,cutoff=widgets.IntSlider(min=1,max=100,step=1,value=20,continuous_update=False,))

interactive(children=(Dropdown(description='Bilde: ', disabled=True, options={'Saturn Med støy': 'cassini-inte…

Som dere kan se er mesteparten av sinussignalet forsvunnet.
I bilde 3 kan dere se filteret som ble produset, fiteret ble justert ved å prøve flere verdier av cutoff til det jeg synes er det beste resultatet ble produsert. 